# Get universitary restaurants menus



<a href="4_Regex.html">Go back to Chapter 3</a> 

- First import module needed

In [1]:
from bs4 import BeautifulSoup
import urllib3
import re
import time
from collections import defaultdict
import numpy as np
import tqdm

- Select one page as an example, here we use gallia's restaurant


In [2]:
urlpage = 'https://www.crous-strasbourg.fr/restaurant/resto-u-esplanade/'
urlpage = 'https://www.crous-strasbourg.fr/restaurant/resto-u-gallia/'
urlpage

'https://www.crous-strasbourg.fr/restaurant/resto-u-gallia/'

- Get the html content using Beautiful soup

In [3]:
req = urllib3.PoolManager()
res = req.request('GET', urlpage)
soup = BeautifulSoup(res.data, 'html.parser')
str(soup)[:200000]

'<!DOCTYPE html>\n\n<html lang="fr-FR" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="utf-8"/>\n<title>Resto U\' Gallia - Crous Strasbourg</title>\n<meta content="width=device-width, initial-scale=1.0,user-scalable=no,minimal-ui" name="viewport"/>\n<meta content="width = device-width, initial-scale = 1.0, user-scalable = no,minimal-ui" name="viewport"/>\n<script>                function apbct_attach_event_handler__backend(elem, event, callback) {                    if(typeof window.addEventListener === "function") elem.addEventListener(event, callback);                    else                                                elem.attachEvent(event, callback);                }                apbct_attach_event_handler__backend(document, \'DOMContentLoaded\', function(){                    if (typeof apbctLocalStorage === "object" && ctPublic.data__key_is_ok) {                        apbctLocalStorage.set(\'ct_checkjs\', \'e68035dbe9fd60f5cdfd8c6d51576f3a79ba36e253329083ca6dc43ab

- Find out how you will get the information you want: with a right click you have previously inspect the webpage with your navigator. ```'menu'``` seems to be a good option to get meals in a structured way.  
```'menu'``` is the html class and ```'div'```is the element. See how it is written ```<div class="menu">```

- Have a look on the content after specifiying the type of element and class you wanted to match. Each 


In [4]:
# Return elements that matched the html class in a list
content = soup.find_all('div' ,class_= 'menu')
content[:1]

[<div class="menu">
 <div class="menu_date">
 <div class="prev"></div>
 <time class="menu_date_title">Menu du lundi 23 janvier 2023</time>
 <div class="next"></div>
 </div>
 <div class="meal">
 <div class="meal_title">Déjeuner</div>
 <ul class="meal_foodies">
 <li>Menu étudiant<ul><li>Cheeseburger</li><li>Frites</li><li>Spaghettis carbonara</li></ul></li><li>Végétarien ou végétal<ul><li>Galette de seitan</li><li>Poêlée Mexicaine</li><li>Frites</li></ul></li><li>Brasserie<ul><li>Côtelette d'agneau jus corsé au thym</li><li>Agneau Origine Irlande</li><li>Poêlée Mexicaine</li><li>Frites </li><li>Pizza royale</li><li>Tarte flambée traditionnelle</li></ul></li><li>Vente à Emporter<ul><li>FORMULE ÉTUDIANTE avec le Parisien et le Maraicher</li><li>Pavé Gandhi</li><li>Le Rodele</li><li>Panini Napoli</li></ul></li>
 </ul>
 </div>
 <div class="meal">
 <div class="meal_title">Dîner</div>
 <ul class="meal_foodies">
 <li>Menu étudiant<ul><li>Poulet yassa</li><li>Petits pois</li><li>Boulghour</li><l

- Let's first create a function to get the webpage, the arguments to get a flexible function could be: 
    - url of the page
    - element of the html
    - class

In [5]:
def get_page(urlpage,element ,html_class):
    """
    From an link and a html class that you select by inspecting manually the page
    Return a list with elements that matched this class
    """
    # Get page in html
    req = urllib3.PoolManager()
    res = req.request('GET', urlpage)
    soup = BeautifulSoup(res.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    content = soup.find_all(element ,class_= html_class)
    
    return content 

In [6]:
contents = get_page(urlpage,
                'div','menu')
content = contents[0]
content

<div class="menu">
<div class="menu_date">
<div class="prev"></div>
<time class="menu_date_title">Menu du lundi 23 janvier 2023</time>
<div class="next"></div>
</div>
<div class="meal">
<div class="meal_title">Déjeuner</div>
<ul class="meal_foodies">
<li>Menu étudiant<ul><li>Cheeseburger</li><li>Frites</li><li>Spaghettis carbonara</li></ul></li><li>Végétarien ou végétal<ul><li>Galette de seitan</li><li>Poêlée Mexicaine</li><li>Frites</li></ul></li><li>Brasserie<ul><li>Côtelette d'agneau jus corsé au thym</li><li>Agneau Origine Irlande</li><li>Poêlée Mexicaine</li><li>Frites </li><li>Pizza royale</li><li>Tarte flambée traditionnelle</li></ul></li><li>Vente à Emporter<ul><li>FORMULE ÉTUDIANTE avec le Parisien et le Maraicher</li><li>Pavé Gandhi</li><li>Le Rodele</li><li>Panini Napoli</li></ul></li>
</ul>
</div>
<div class="meal">
<div class="meal_title">Dîner</div>
<ul class="meal_foodies">
<li>Menu étudiant<ul><li>Poulet yassa</li><li>Petits pois</li><li>Boulghour</li><li>Pâtes aux légu

In [7]:
meals = str(content).split('<div class="meal">')
for meal in meals:
    print(meal)

<div class="menu">
<div class="menu_date">
<div class="prev"></div>
<time class="menu_date_title">Menu du lundi 23 janvier 2023</time>
<div class="next"></div>
</div>


<div class="meal_title">Déjeuner</div>
<ul class="meal_foodies">
<li>Menu étudiant<ul><li>Cheeseburger</li><li>Frites</li><li>Spaghettis carbonara</li></ul></li><li>Végétarien ou végétal<ul><li>Galette de seitan</li><li>Poêlée Mexicaine</li><li>Frites</li></ul></li><li>Brasserie<ul><li>Côtelette d'agneau jus corsé au thym</li><li>Agneau Origine Irlande</li><li>Poêlée Mexicaine</li><li>Frites </li><li>Pizza royale</li><li>Tarte flambée traditionnelle</li></ul></li><li>Vente à Emporter<ul><li>FORMULE ÉTUDIANTE avec le Parisien et le Maraicher</li><li>Pavé Gandhi</li><li>Le Rodele</li><li>Panini Napoli</li></ul></li>
</ul>
</div>


<div class="meal_title">Dîner</div>
<ul class="meal_foodies">
<li>Menu étudiant<ul><li>Poulet yassa</li><li>Petits pois</li><li>Boulghour</li><li>Pâtes aux légumes</li></ul></li><li>Origine de n


- We want everything between ```<time class="menu_date_title">Menu du ``` and the first ```</time>``` that appears, we can just catch the group of character between these two entities. Create a group with ```()``` to extract this part with ```re.findall``` :
    - This group can be composed of everything so specify a ```.```
    - We can have 0 or more charater so put a ```*```after the ```.```
    - Finally the ```?``` will control for the repetition ```</time>```, so when it will meet the first ```</time>```, it will stop.
 
- You will then try to find if there is some stuff in your string that match this condition. The regex will be written ```'<time class="menu_date_title">Menu du (.*?)</time>'```. Of course it doen't mean there are no other way of doing it or even writting it, but it works and it's pretty simple to read.


In [8]:
date = re.findall('<time class="menu_date_title">Menu du (.*?)</time>',str(content))

print(date)

['lundi 23 janvier 2023']


In [9]:
meals = meals[1:]
for meal in meals:
    print(meal)


<div class="meal_title">Déjeuner</div>
<ul class="meal_foodies">
<li>Menu étudiant<ul><li>Cheeseburger</li><li>Frites</li><li>Spaghettis carbonara</li></ul></li><li>Végétarien ou végétal<ul><li>Galette de seitan</li><li>Poêlée Mexicaine</li><li>Frites</li></ul></li><li>Brasserie<ul><li>Côtelette d'agneau jus corsé au thym</li><li>Agneau Origine Irlande</li><li>Poêlée Mexicaine</li><li>Frites </li><li>Pizza royale</li><li>Tarte flambée traditionnelle</li></ul></li><li>Vente à Emporter<ul><li>FORMULE ÉTUDIANTE avec le Parisien et le Maraicher</li><li>Pavé Gandhi</li><li>Le Rodele</li><li>Panini Napoli</li></ul></li>
</ul>
</div>


<div class="meal_title">Dîner</div>
<ul class="meal_foodies">
<li>Menu étudiant<ul><li>Poulet yassa</li><li>Petits pois</li><li>Boulghour</li><li>Pâtes aux légumes</li></ul></li><li>Origine de nos viandes<ul><li>Volaille Origine France</li></ul></li>
</ul>
</div>
</div>


In [10]:
types =  re.findall('<div class="meal_title">(.*?)</div>',str(content))
meal_foodies = meals[0].split('</ul></li>')[:-1]
meal_foodies

['\n<div class="meal_title">Déjeuner</div>\n<ul class="meal_foodies">\n<li>Menu étudiant<ul><li>Cheeseburger</li><li>Frites</li><li>Spaghettis carbonara</li>',
 '<li>Végétarien ou végétal<ul><li>Galette de seitan</li><li>Poêlée Mexicaine</li><li>Frites</li>',
 "<li>Brasserie<ul><li>Côtelette d'agneau jus corsé au thym</li><li>Agneau Origine Irlande</li><li>Poêlée Mexicaine</li><li>Frites </li><li>Pizza royale</li><li>Tarte flambée traditionnelle</li>",
 '<li>Vente à Emporter<ul><li>FORMULE ÉTUDIANTE avec le Parisien et le Maraicher</li><li>Pavé Gandhi</li><li>Le Rodele</li><li>Panini Napoli</li>']

- It's still very dirty down there, let's clean it up!

- Let's retrieve everything within ```<li></li>```.

In [11]:
pole, meal = meal_foodies[0].split('<ul>')
print(meal)

<li>Cheeseburger</li><li>Frites</li><li>Spaghettis carbonara</li>


In [12]:
print(pole)
print('-----')
pole = re.search('<li>(.+)',pole).group(1)
print(pole)


<div class="meal_title">Déjeuner</div>
<ul class="meal_foodies">
<li>Menu étudiant
-----
Menu étudiant


In [13]:
details = re.findall('<li>(.*?)</li>',
                   meal)
details  

['Cheeseburger', 'Frites', 'Spaghettis carbonara']

In [14]:
pole_menu = dict()
for meal_foodie in meal_foodies:
    pole, meal = meal_foodie.split('<ul>')

    pole = re.search('<li>(.+)',pole).group(1)
    details = re.findall('<li>(.*?)</li>',
                   meal)
    pole_menu.update({pole:details})

In [15]:
pole_menu

{'Menu étudiant': ['Cheeseburger', 'Frites', 'Spaghettis carbonara'],
 'Végétarien ou végétal': ['Galette de seitan', 'Poêlée Mexicaine', 'Frites'],
 'Brasserie': ["Côtelette d'agneau jus corsé au thym",
  'Agneau Origine Irlande',
  'Poêlée Mexicaine',
  'Frites ',
  'Pizza royale',
  'Tarte flambée traditionnelle'],
 'Vente à Emporter': ['FORMULE ÉTUDIANTE avec le Parisien et le Maraicher',
  'Pavé Gandhi',
  'Le Rodele',
  'Panini Napoli']}

In [16]:
content = contents[0]
meals = str(content).split('<div class="meal">')
meals = meals[1:]
date = re.findall('<time class="menu_date_title">Menu du (.*?)</time>',str(content))

all_meals = dict()
for meal in meals:
    pole_menu = dict()
    meal_foodies = meal.split('</ul></li>')[:-1]
    meal_type = re.findall('<div class="meal_title">(.*?)</div>',meal)[0]
    for meal_foodie in meal_foodies:
        pole, foodie = meal_foodie.split('<ul>')

        pole = re.search('<li>(.+)',pole).group(1)

        details = re.findall('<li>(.*?)</li>',
                       foodie)
        pole_menu.update({pole:details})

    all_meals.update({meal_type:pole_menu})

all_meals

{'Déjeuner': {'Menu étudiant': ['Cheeseburger',
   'Frites',
   'Spaghettis carbonara'],
  'Végétarien ou végétal': ['Galette de seitan', 'Poêlée Mexicaine', 'Frites'],
  'Brasserie': ["Côtelette d'agneau jus corsé au thym",
   'Agneau Origine Irlande',
   'Poêlée Mexicaine',
   'Frites ',
   'Pizza royale',
   'Tarte flambée traditionnelle'],
  'Vente à Emporter': ['FORMULE ÉTUDIANTE avec le Parisien et le Maraicher',
   'Pavé Gandhi',
   'Le Rodele',
   'Panini Napoli']},
 'Dîner': {'Menu étudiant': ['Poulet yassa',
   'Petits pois',
   'Boulghour',
   'Pâtes aux légumes'],
  'Origine de nos viandes': ['Volaille Origine France']}}

- We know now two things :
   - Download the content of each meal and store it
   - Clean up individually the list to get names and meals for a given restaurant.
   
- let's create a function!
   - Get the each meal and each name for a given restaurant and for a given schedule, we just need the whole content of this given meal.  

In [17]:
def get_details(meal_foodie):
    """
    Get meal content and pole name
    """
    # Separate name and food
    pole, foodie = meal_foodie.split('<ul>')
    
    # Extract meals keeping everything between <li> and </li>
    details = re.findall('<li>(.*?)</li>',
                       foodie)
    
    # Clean name
    pole = re.search('<li>(.+)',pole).group(1)
    
    return pole, details

In [18]:
def get_pole_menu(meal_foodies):
    """
    Iterate over each pole and store the information about the meal 
    """
    pole_menu = dict() 
    # Iterate over each possble choice
    for meal_foodie in meal_foodies:
        pole, details = get_details(meal_foodie)
        # Update the dictionary
        pole_menu.update({pole: details})
    return pole_menu

In [19]:
def clean_meals(content):
    """
    Separate lunch and dinner, remove first element as it does not contains information about food
    """
    meals = str(content).split('<div class="meal">')
    meals = meals[1:]
    return meals

In [20]:
def get_date(content):
    """
    Get date
    """
    date = re.findall('<time class="menu_date_title">Menu du (.*?)</time>',str(content))[0]
    return date

In [21]:
def get_meal_type(meal):
    """
    Are you having lunch or dinner?
    """
    meal_type = re.findall('<div class="meal_title">(.*?)</div>',meal)[0]
    return meal_type

In [22]:
def get_all_meals_for_given_date(content):
    """
    Put all the functions together, return date and all meal for a given restaurant at a gevien date
    """
    date = get_date(content)
    meals = clean_meals(content)
    
    all_meals = dict()
    # Iterate over each meal
    for meal in meals:
        meal_foodies = meal.split('</ul></li>')[:-1]
        meal_type = get_meal_type(meal)
        pole_menu = get_pole_menu(meal_foodies)
        
        # Store the resulte
        all_meals.update({meal_type:pole_menu})
        
    return date ,all_meals 

In [23]:
get_all_meals_for_given_date(contents[0])

('lundi 23 janvier 2023',
 {'Déjeuner': {'Menu étudiant': ['Cheeseburger',
    'Frites',
    'Spaghettis carbonara'],
   'Végétarien ou végétal': ['Galette de seitan',
    'Poêlée Mexicaine',
    'Frites'],
   'Brasserie': ["Côtelette d'agneau jus corsé au thym",
    'Agneau Origine Irlande',
    'Poêlée Mexicaine',
    'Frites ',
    'Pizza royale',
    'Tarte flambée traditionnelle'],
   'Vente à Emporter': ['FORMULE ÉTUDIANTE avec le Parisien et le Maraicher',
    'Pavé Gandhi',
    'Le Rodele',
    'Panini Napoli']},
  'Dîner': {'Menu étudiant': ['Poulet yassa',
    'Petits pois',
    'Boulghour',
    'Pâtes aux légumes'],
   'Origine de nos viandes': ['Volaille Origine France']}})

In [24]:
for content in contents:
    print(get_all_meals_for_given_date(content))

('lundi 23 janvier 2023', {'Déjeuner': {'Menu étudiant': ['Cheeseburger', 'Frites', 'Spaghettis carbonara'], 'Végétarien ou végétal': ['Galette de seitan', 'Poêlée Mexicaine', 'Frites'], 'Brasserie': ["Côtelette d'agneau jus corsé au thym", 'Agneau Origine Irlande', 'Poêlée Mexicaine', 'Frites ', 'Pizza royale', 'Tarte flambée traditionnelle'], 'Vente à Emporter': ['FORMULE ÉTUDIANTE avec le Parisien et le Maraicher', 'Pavé Gandhi', 'Le Rodele', 'Panini Napoli']}, 'Dîner': {'Menu étudiant': ['Poulet yassa', 'Petits pois', 'Boulghour', 'Pâtes aux légumes'], 'Origine de nos viandes': ['Volaille Origine France']}})
('mardi 24 janvier 2023', {'Déjeuner': {'Menu étudiant': ['MENU ALLEMAND', 'Bratwurst ( Saucisse grillée)', 'Choucroute au cumin', 'Pommes de terre sautées'], 'Végétarien ou végétal': ['MENU ALLEMAND', 'Leipziger Allerlei', "(Mélange de légumes poêlées à l'émincé végétal)", 'Choucroute au cumin', 'Pommes de terre sautées'], 'Brasserie': ['Potatoes burger', 'Frites de patates do

- Now we need to get all restaurants urls, we will them here :
 https://www.crous-strasbourg.fr/se-restaurer/ou-manger/

- Use the get_page function, specify ```'article'``` and ```''``` because we want to catch up ```<article>```.


In [25]:
urlpage = 'https://www.crous-strasbourg.fr/se-restaurer/ou-manger/'
   
links = get_page(urlpage,'article','')
links[:5]

[<article>
 <a href="https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r/">
 <div class="restaurant_title">Cafétéria Mini R</div> <div class="restaurant_area">Strasbourg</div> </a>
 </article>,
 <article>
 <a href="https://www.crous-strasbourg.fr/restaurant/le-32/">
 <div class="restaurant_title">Le 32</div> <div class="restaurant_area">Strasbourg</div> </a>
 </article>,
 <article>
 <a href="https://www.crous-strasbourg.fr/restaurant/cafeteria-de-la-fac-de-droit/">
 <div class="restaurant_title">Cafétéria de la Fac de Droit</div> <div class="restaurant_area">Strasbourg</div> </a>
 </article>,
 <article>
 <a href="https://www.crous-strasbourg.fr/restaurant/lannexe/">
 <div class="restaurant_title">L’Annexe</div> <div class="restaurant_area">Strasbourg</div> </a>
 </article>,
 <article>
 <a href="https://www.crous-strasbourg.fr/restaurant/resto-u-de-colmar/">
 <div class="restaurant_title">Resto U’ de Colmar</div> <div class="restaurant_area">Colmar</div> </a>
 </article>]

- We see first that links that are interesting contains restaurant. That's great because it's written in the url!

- Loop over links' list, check if we match ```restaurant``` in the url.
   - Capture the name of the restaurant and the appropriate link
   - Store it in a dict

In [26]:
cleaned_links = {}
for link in links:
    name = re.search('<div class="restaurant_title">(.*?)</div>',str(link)).group(1)
    link = re.search('<a href="(.*?)">',str(link)).group(1)
    if 'restaurant/' in link:
        cleaned_links.update({name:link})
cleaned_links

{'Cafétéria Mini R': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r/',
 'Le 32': 'https://www.crous-strasbourg.fr/restaurant/le-32/',
 'Cafétéria de la Fac de Droit': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-de-la-fac-de-droit/',
 'L’Annexe': 'https://www.crous-strasbourg.fr/restaurant/lannexe/',
 'Resto U’ de Colmar': 'https://www.crous-strasbourg.fr/restaurant/resto-u-de-colmar/',
 'Le Cristal Shop (RU Esplanade)': 'https://www.crous-strasbourg.fr/restaurant/le-cristal-shop-ru-esplanade/',
 'Cafétéria Illkirch': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-illkirch/',
 'Kiosque (RU Esplanade)': 'https://www.crous-strasbourg.fr/restaurant/kiosque-ru-esplanade/',
 'Resto U’ de l’Illberg': 'https://www.crous-strasbourg.fr/restaurant/resto-u-de-lillberg/',
 'Resto U’ de l’IUT Mulhouse': 'https://www.crous-strasbourg.fr/restaurant/resto-u-de-liut-mulhouse/',
 'Cafétéria de la Fonderie': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-de-la-fonderi

- Let's put it in a function with no argument

In [27]:
def get_all_links():
    """
    Get all restaurants links and create a dict of link with ru_names as keys
    """
    # Webpage from which we extract all links
    urlpage = 'https://www.crous-strasbourg.fr/se-restaurer/ou-manger/'
   
    # Use our get_page function to extract a list with all list
    links = get_page(urlpage,'article','')
    
    # Create an empty dict
    names_links = {}
    
    # Iterate over each link
    for link in links:
        # Check if the restaurant is open, it is written in the url !
        if bool(re.search('/restaurant/',str(link))):
            # Extract the name of the restaurant
            name = re.search('<div class="restaurant_title">(.*?)</div>',str(link)).group(1)
            # Extract the link
            link = re.search('<a href="(.*?)">',str(link)).group(1)
            # Update the dict
            names_links.update({name : link})
            
    return names_links
    

In [28]:
get_all_links()

{'Cafétéria Mini R': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-mini-r/',
 'Le 32': 'https://www.crous-strasbourg.fr/restaurant/le-32/',
 'Cafétéria de la Fac de Droit': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-de-la-fac-de-droit/',
 'L’Annexe': 'https://www.crous-strasbourg.fr/restaurant/lannexe/',
 'Resto U’ de Colmar': 'https://www.crous-strasbourg.fr/restaurant/resto-u-de-colmar/',
 'Le Cristal Shop (RU Esplanade)': 'https://www.crous-strasbourg.fr/restaurant/le-cristal-shop-ru-esplanade/',
 'Cafétéria Illkirch': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-illkirch/',
 'Kiosque (RU Esplanade)': 'https://www.crous-strasbourg.fr/restaurant/kiosque-ru-esplanade/',
 'Resto U’ de l’Illberg': 'https://www.crous-strasbourg.fr/restaurant/resto-u-de-lillberg/',
 'Resto U’ de l’IUT Mulhouse': 'https://www.crous-strasbourg.fr/restaurant/resto-u-de-liut-mulhouse/',
 'Cafétéria de la Fonderie': 'https://www.crous-strasbourg.fr/restaurant/cafeteria-de-la-fonderi

- It seems that everything works fine, let's put all together!

- The basis of our final step will be as follow:
   - Get all links
   - Iterate on it

In [29]:
links = get_all_links()


- Now we just need to think about how to store it: a dictionary is a good solution. Once each date is set, we just need to update the dictionary with the next restaurant! To do it without checking if the key we want to update already exists, we can use a defaultdict from the collections module.

In [30]:
all_dates_all_menus = defaultdict(dict)
for name in tqdm.tqdm(links):
    contents = get_page(links[name],
                    'div','menu')
    for content in contents:
        date ,all_meals = get_all_meals_for_given_date(content)
        all_dates_all_menus[date].update({name:all_meals})
        
all_dates_all_menus

100%|██████████| 23/23 [00:14<00:00,  1.61it/s]


defaultdict(dict,
            {'lundi 23 janvier 2023': {'Cafétéria Mini R': {'Déjeuner': {'Sandwichs': ['PARISIEN',
                 'MARAÎCHER',
                 'RUSTIQUE',
                 'MANHATTAN'],
                'SOUPE': ['VELOUTÉ DE CHOUX FLEURS'],
                'Plat chaud': ['PAVÉ DE POISSON BORDELAISE',
                 'RIZ AUX PETITS LÉGUMES'],
                'Chaud et salé': ['BOX SPAGHETTI BOLOGNAISE'],
                'Desserts': ['COOKIE 3 CHOCOLATS',
                 'SALADE DE FRUITS',
                 'FROMAGE BLANC AUX MYRTILLES'],
                'Yaourts': ["SAVEUR D'AUTREFOIS VANILLE",
                 'GOURMAND CHOCOLAT',
                 'NATURE']}},
              'Le 32': {'Déjeuner': {'Carte du jour au 32 - Plats': ['Suprême de merlu, sauce au beurre blanc nantais',
                 '-',
                 'Poêlée de butternuts aux pommes et aux noix',
                 'Garnitures',
                 'Pommes de terre grenailles',
                 'Jardin

- Finally! You can easily get information from your dict

In [31]:
all_dates_all_menus['mercredi 25 janvier 2023']['Resto U’ Gallia']['Déjeuner']

{'Menu étudiant': ['Escalope Viennoise',
  'Trio de légumes',
  'Pommes rissolées'],
 'Végétarien ou végétal': ['Lasagnes végétariennes', 'Salade verte'],
 'Brasserie': ["Magret de canard à l'orange",
  'Trio de légumes',
  'Pommes rissolées',
  'Pizza chorizo et maïs',
  'Tarte flambée au saumon'],
 'Vente à Emporter': ['FORMULE ÉTUDIANTE avec le Parisien et le Maraicher',
  'Pavé Gandhi',
  'Le Rodele',
  'Enchilada légumes et chèvre']}

- Now we can finish with a small user interface to make things more fancy !
- Remerber that we saw an easy way of formatting strings, create a function to print out stuff with an index aside

In [32]:
def show_item(list_):
    i = 1
    for item in list_:
        print('%s : %s'%(str(i),item))
        i+=1

- Ask the user the rest using ```input()```

In [34]:
quit = 'N'
change_rest = 'Y'
while quit=='N':
    dates = list(all_dates_all_menus.keys())
    show_item(dates)
    date_choosed = int(input('Select the date (type the number aside)'))
    restaurants = list(all_dates_all_menus[dates[date_choosed-1]].keys())
    while change_rest == 'Y':
        
        show_item(restaurants)
        rest_choosed = int(input('Select the restaurant (type the number aside)'))
        meal_keys = list(all_dates_all_menus[dates[date_choosed-1]][restaurants[rest_choosed-1]].keys())
        show_item(meal_keys)
        meal_choosed = int(input('Select the meal (type the number aside)'))
        
        menu = all_dates_all_menus[dates[date_choosed-1]][restaurants[rest_choosed-1]][meal_keys[meal_choosed-1]]
        for meal in menu:
            print('''%s :
            %s
              '''%(meal,menu[meal]))
        change_rest = input('Do you want to select another restaurant ? (Y/N)')
    
    
    quit = input('Do you want to quit ? (Y/N)')
    if quit == 'N':
        change_rest = 'Y'

1 : lundi 23 janvier 2023
2 : mardi 24 janvier 2023
3 : mercredi 25 janvier 2023
4 : jeudi 26 janvier 2023
5 : vendredi 27 janvier 2023
6 : lundi 30 janvier 2023
7 : mardi 31 janvier 2023
8 : mercredi 1 février 2023
9 : jeudi 2 février 2023
10 : vendredi 3 février 2023
Select the date (type the number aside)1
1 : Cafétéria Mini R
2 : Le 32
3 : L’Annexe
4 : Resto U’ de Colmar
5 : Le Cristal Shop (RU Esplanade)
6 : Kiosque (RU Esplanade)
7 : Resto U’ de l’Illberg
8 : Resto U’ de l’IUT Mulhouse
9 : Cafétéria de la Fonderie
10 : Resto U’ Cronenbourg
11 : Resto U’ Illkirch
12 : Cafétéria Le PEGE
13 : Learning Center
14 : Resto U’ Esplanade
15 : Resto U’ Paul Appell
16 : Cybercafé (RU Illberg)
17 : Resto U’ Gallia
Select the restaurant (type the number aside)12
1 : Déjeuner
Select the meal (type the number aside)1
Entrées   Desserts :
            ['Crudités du jour', 'Salade de jambon', 'Thon &amp; crudités', 'Oeufs &amp; crudités ', 'Tartelette citron', 'Religieuse chocolat', 'Tiramisu', 'C

# Put everything together

## Import modules

In [35]:
# To scrap 
from bs4 import BeautifulSoup
import urllib3

# To handle text manipulation
import re

# to handle time
import time

# to use defaultdict object 
from collections import defaultdict

# To have a nice graphical bar that shows where we are in a process
import tqdm

## Get all pages to scrap

### Functions

In [36]:
def get_page(urlpage,element ,html_class):
    """
    From an link and a html class that you select by inspecting manually the page
    Return a list with elements that matched this class
    """
    # Get page in html
    req = urllib3.PoolManager()
    res = req.request('GET', urlpage)
    soup = BeautifulSoup(res.data, 'html.parser')
    
    # Return elements that matched the html class in a list
    content = soup.find_all(element ,class_= html_class)
    
    return content 

def get_all_links():
    """
    Get all restaurants links and create a dict of link with ru_names as keys
    """
    # Webpage from which we extract all links
    urlpage = 'https://www.crous-strasbourg.fr/se-restaurer/ou-manger/'
   
    # Use our get_page function to extract a list with all list
    links = get_page(urlpage,'article','')
    
    # Create an empty dict
    names_links = {}
    
    # Iterate over each link
    for link in links:
        # Check if the restaurant is open, it is written in the url !
        if bool(re.search('/restaurant/',str(link))):
            # Extract the name of the restaurant
            name = re.search('<div class="restaurant_title">(.*?)</div>',str(link)).group(1)
            # Extract the link
            link = re.search('<a href="(.*?)">',str(link)).group(1)
            # Update the dict
            names_links.update({name : link})
            
    return names_links
    

### Execution

In [37]:
links = get_all_links()

## Get all menus from all restaurants

### Functions

In [38]:
def get_details(meal_foodie):
    """
    Get meal content and pole name
    """
    # Separate name and food
    pole, foodie = meal_foodie.split('<ul>')
    
    # Extract meals keeping everything between <li> and </li>
    details = re.findall('<li>(.*?)</li>',
                       foodie)
    
    # Clean name
    pole = re.search('<li>(.+)',pole).group(1)
    
    return pole, details

def get_pole_menu(meal_foodies):
    """
    Iterate over each pole and store the information about the meal 
    """
    pole_menu = dict() 
    # Iterate over each possble choice
    for meal_foodie in meal_foodies:
        pole, details = get_details(meal_foodie)
        # Update the dictionary
        pole_menu.update({pole: details})
    return pole_menu

def clean_meals(content):
    """
    Separate lunch and dinner, remove first element as it does not contains information about food
    """
    meals = str(content).split('<div class="meal">')
    meals = meals[1:]
    return meals

def get_date(content):
    """
    Get date
    """
    date = re.findall('<time class="menu_date_title">Menu du (.*?)</time>',str(content))[0]
    return date

def get_meal_type(meal):
    """
    Are you having lunch or dinner?
    """
    meal_type = re.findall('<div class="meal_title">(.*?)</div>',meal)[0]
    return meal_type

def get_all_meals_for_given_date(content):
    """
    Put all the functions together, return date and all meal for a given restaurant at a gevien date
    """
    date = get_date(content)
    meals = clean_meals(content)
    
    all_meals = dict()
    # Iterate over each meal
    for meal in meals:
        meal_foodies = meal.split('</ul></li>')[:-1]
        meal_type = get_meal_type(meal)
        pole_menu = get_pole_menu(meal_foodies)
        
        # Store the resulte
        all_meals.update({meal_type:pole_menu})
        
    return date ,all_meals 

### Execution

In [39]:
all_dates_all_menus = defaultdict(dict)
for name in tqdm.tqdm(links):
    contents = get_page(links[name],
                    'div','menu')
    for content in contents:
        date ,all_meals = get_all_meals_for_given_date(content)
        all_dates_all_menus[date].update({name:all_meals})
        
all_dates_all_menus

100%|██████████| 23/23 [00:20<00:00,  1.12it/s]


defaultdict(dict,
            {'lundi 23 janvier 2023': {'Cafétéria Mini R': {'Déjeuner': {'Sandwichs': ['PARISIEN',
                 'MARAÎCHER',
                 'RUSTIQUE',
                 'MANHATTAN'],
                'SOUPE': ['VELOUTÉ DE CHOUX FLEURS'],
                'Plat chaud': ['PAVÉ DE POISSON BORDELAISE',
                 'RIZ AUX PETITS LÉGUMES'],
                'Chaud et salé': ['BOX SPAGHETTI BOLOGNAISE'],
                'Desserts': ['COOKIE 3 CHOCOLATS',
                 'SALADE DE FRUITS',
                 'FROMAGE BLANC AUX MYRTILLES'],
                'Yaourts': ["SAVEUR D'AUTREFOIS VANILLE",
                 'GOURMAND CHOCOLAT',
                 'NATURE']}},
              'Le 32': {'Déjeuner': {'Carte du jour au 32 - Plats': ['Suprême de merlu, sauce au beurre blanc nantais',
                 '-',
                 'Poêlée de butternuts aux pommes et aux noix',
                 'Garnitures',
                 'Pommes de terre grenailles',
                 'Jardin

## User Interface

### Functions

In [40]:
def show_item(list_):
    i = 1
    for item in list_:
        print('%s : %s'%(str(i),item))
        i+=1

### Execution

In [41]:
quit = 'N'
change_rest = 'Y'
while quit=='N':
    dates = list(all_dates_all_menus.keys())
    show_item(dates)
    date_choosed = int(input('Select the date (type the number aside)'))
    restaurants = list(all_dates_all_menus[dates[date_choosed-1]].keys())
    while change_rest == 'Y':
        
        show_item(restaurants)
        rest_choosed = int(input('Select the restaurant (type the number aside)'))
        meal_keys = list(all_dates_all_menus[dates[date_choosed-1]][restaurants[rest_choosed-1]].keys())
        show_item(meal_keys)
        meal_choosed = int(input('Select the meal (type the number aside)'))
        
        menu = all_dates_all_menus[dates[date_choosed-1]][restaurants[rest_choosed-1]][meal_keys[meal_choosed-1]]
        for meal in menu:
            print('''%s :
            %s
              '''%(meal,menu[meal]))
        change_rest = input('Do you want to select another restaurant ? (Y/N)')
    
    
    quit = input('Do you want to quit ? (Y/N)')
    if quit == 'N':
        change_rest = 'Y'

1 : lundi 23 janvier 2023
2 : mardi 24 janvier 2023
3 : mercredi 25 janvier 2023
4 : jeudi 26 janvier 2023
5 : vendredi 27 janvier 2023
6 : lundi 30 janvier 2023
7 : mardi 31 janvier 2023
8 : mercredi 1 février 2023
9 : jeudi 2 février 2023
10 : vendredi 3 février 2023
Select the date (type the number aside)3
1 : Cafétéria Mini R
2 : Le 32
3 : Resto U’ de Colmar
4 : Le Cristal Shop (RU Esplanade)
5 : Kiosque (RU Esplanade)
6 : Resto U’ de l’Illberg
7 : Resto U’ de l’IUT Mulhouse
8 : Cafétéria de la Fonderie
9 : Resto U’ Cronenbourg
10 : Resto U’ Illkirch
11 : Learning Center
12 : Resto U’ Esplanade
13 : Cybercafé (RU Illberg)
14 : Resto U’ Gallia
Select the restaurant (type the number aside)6
1 : Déjeuner
Select the meal (type the number aside)1
Menu étudiant - Tarif social :
            ['menu franco allemand', 'Soupe à la bière', 'Fricadelle sauce crème champignons', 'Spaetzle', 'Petits pois', 'Apfelstrudel']
              
Menu végétarien - Tarif social :
            ['Tarte à la ch